In [37]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical


result = pd.read_csv('preprocessed_processed_image.csv')
print(result)


        선종  상태  state  vessel_type  nation_type
0     낚시어선  조업      0            8            1
1      범장망  조업      0            3            0
2     낚시어선  표류      2            8            1
3     등광조망  이동      1            2            0
4     등광조망  표류      2            2            0
...    ...  ..    ...          ...          ...
1576   안강망  조업      0            4            1
1577    연승  표류      2            5            1
1578  등광조망  조업      0            2            0
1579  등광조망  조업      0            2            0
1580  등광조망  이동      1            2            0

[1581 rows x 5 columns]


In [29]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('mobilenet_state_model.h5')

In [40]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical

# 이미지 디렉토리 경로
image_dir = 'image'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 레이블을 적절히 설정하세요
labels = result['state']
labels = to_categorical(labels, num_classes=3)



In [41]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def MobileNetV2(input_shape=(256, 256, 3), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2')
    return model

model = MobileNetV2()
model.summary()



/tmp/ipykernel_102634/4152683701.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_9      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 2,388,227 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [45]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [46]:
tf.random.set_seed(42)
history = model.fit(images, labels, epochs=20, verbose=1)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.3747 - loss: 1.2590
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.3943 - loss: 1.0711
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4395 - loss: 1.0567
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4343 - loss: 1.0409
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4230 - loss: 1.0548
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4127 - loss: 1.0320
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4745 - loss: 1.0177
Epoch 8/20
10/50 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.4021 - loss: 1.0617

KeyboardInterrupt: 

In [1]:
# 모델 저장
model.save('mobilenet_state_model.h5')  # HDF5 형식으로 저장




NameError: name 'model' is not defined

In [36]:
prediction = model.predict(images)
# binary_predictions = (prediction > 0.5).astype(int)
class_prediction = np.argmax(prediction, axis=1)

print(class_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [26]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical

# 이미지 디렉토리 경로
image_dir = 'mini_test'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)
